In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import ast

######### Get the files from gs://tpu-cg-us/ranker_isp_golden_nu_Hindi/ ###############################
df2 = pd.read_csv('000000000008',sep='&',header=None)
df3 = pd.read_csv('000000000009',sep='&',header=None)
df4 = pd.read_csv('000000000010',sep='&',header=None)
df = pd.concat([df2,df3,df4])

df.columns = ['postlang','time','userId','postId','eng_tags_mask','post_dense_features','post_embed','sparse_features_eng_tags',
'sparse_features_tagId','sparse_features_userDistrict','time_dayofweek','time_hour','time_is_afternoon','time_is_evening',
'time_is_morning','time_is_night','time_is_weekend','user_dense_features','user_embed','isp_date','model','isp_score','is_like',
'is_share','is_fav','is_vp_succ','is_vp_skip']

df['user_embed2']= df['user_embed'].apply(lambda x:list(map(lambda x:float(x),ast.literal_eval(x))))
df['post_embed2']= df['post_embed'].apply(lambda x:list(map(lambda x:float(x),ast.literal_eval(x))))

def func(row):
    return np.dot(normalize(np.reshape(np.array(row['user_embed2'],dtype=np.float32),(1,-1)),norm='l2'),
                  normalize(np.reshape(np.array(row['post_embed2'],dtype=np.float32),(1,-1)),norm='l2').T)[0][0]

df['isp_score2'] = df.apply(func,axis=1)

print("================================")

df.isp_score.fillna(df['isp_score2'], inplace=True)
df.drop(['user_embed2','post_embed2','isp_score2'],axis=1,inplace=True)


df.to_csv('combined_ranker_isp_Hindi_nu.csv',sep='&',index=False)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import numpy as np
import ast
import tensorflow as tf

In [ ]:
df = pd.read_csv("combined_ranker_isp_Hindi_nu.csv",sep='&')

In [ ]:
df.head()

,postlang,time,userId,postId,eng_tags_mask,post_dense_features,post_embed,sparse_features_eng_tags,sparse_features_tagId,sparse_features_userDistrict,...,user_dense_features,user_embed,isp_date,model,isp_score,is_like,is_share,is_fav,is_vp_succ,is_vp_skip
0,Hindi,2023-02-17 00:58:53.948 UTC,1759974102,5443143635,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1","-0.49157286,-0.015197761,0.20486306,0.4278766,...","-0.49157286,-0.015197761,0.20486306,0.4278766,...","3970868,392705,97209,3965189,437743,2859806,10...",3752926,NaN,...,"0.09303941,0.38944566,-0.12592548,0.013801534,...","0.09303941,0.38944566,-0.12592548,0.013801534,...",2023-02-17 00:52:45 UTC,isp_ranker_sugg_video_realtime_ffm,5.228948,NaN,NaN,NaN,0,1
1,Hindi,2023-02-17 00:52:25.496 UTC,2245741155,5143193635,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0","-0.26532394,0.023439672,-0.35637814,-0.1182885...","-0.26532394,0.023439672,-0.35637814,-0.1182885...","851190,457799,4325675,1307875,339940,3970882,9...",173229,NaN,...,"-0.29729316,-0.2076694,-0.19222566,-0.10918815...","-0.29729316,-0.2076694,-0.19222566,-0.10918815...",NaN,NaN,-0.605581,NaN,NaN,NaN,0,1
2,Hindi,2023-02-17 00:44:20.194 UTC,1637723133,7715079135,"1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0","0.016881194,-0.134322,0.48357236,0.51583666,-0...","0.016881194,-0.134322,0.48357236,0.51583666,-0...","6126496,4344180,337474,3965189,3757983,7728706...",718909,NaN,...,"0.12224271,-0.00018345535,-0.13967116,0.192661...","0.12224271,-0.00018345535,-0.13967116,0.192661...",NaN,NaN,-0.774085,NaN,NaN,NaN,0,1
3,Hindi,2023-02-17 00:48:35.506 UTC,855736416,9095229335,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1","0.23273607,-0.21051688,-0.022401525,0.2591531,...","0.23273607,-0.21051688,-0.022401525,0.2591531,...","687,109358,592007,312374,1278163,3970849,39708...",12111522,NaN,...,"0.51846147,-0.028721439,-0.37994006,-0.0895136...","0.51846147,-0.028721439,-0.37994006,-0.0895136...",NaN,NaN,-0.437082,NaN,NaN,NaN,0,1
4,Hindi,2023-02-17 00:33:51.362 UTC,668895138,5847111635,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0","-0.56743217,-0.2478295,-0.531914,-0.22087942,-...","-0.56743217,-0.2478295,-0.531914,-0.22087942,-...","221065,418220,1352256,1173370,2859806,8099565,...",10837639,NaN,...,"0.2939948,0.0075860075,0.07450469,0.05363995,0...","0.2939948,0.0075860075,0.07450469,0.05363995,0...",NaN,NaN,-0.422589,NaN,NaN,NaN,0,1


In [ ]:
df['user_dense_features'].fillna("0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0",inplace=True)
df['sparse_features_eng_tags'].fillna("0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0",inplace=True)
df['post_dense_features'].fillna("0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0",inplace=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['sparse_features_tagId'] = le.fit_transform(df.sparse_features_tagId.values)

le2 = LabelEncoder()
df['sparse_features_userDistrict'] = le2.fit_transform(df.sparse_features_userDistrict.values)

In [ ]:
df['sparse_features_eng_tags2']= df['sparse_features_eng_tags'].apply(lambda x:list(map(lambda x:int(x),ast.literal_eval(x))))


In [ ]:
st=set()
for i in df['sparse_features_eng_tags2']:
  for val in i:
    st.add(val)

In [ ]:
le3 = LabelEncoder()
le3.fit(np.array(list(st)))

li = []
for i in df['sparse_features_eng_tags2']:
  li.append(le3.transform(i))

df['sparse_features_eng_tags3'] = li

In [ ]:
df.dtypes

postlang                         object
time                             object
userId                            int64
postId                            int64
eng_tags_mask                    object
post_dense_features              object
post_embed                       object
sparse_features_eng_tags         object
sparse_features_tagId             int64
sparse_features_userDistrict      int64
time_dayofweek                    int64
time_hour                         int64
time_is_afternoon                 int64
time_is_evening                   int64
time_is_morning                   int64
time_is_night                     int64
time_is_weekend                   int64
user_dense_features              object
user_embed                       object
isp_date                         object
model                            object
isp_score                       float64
is_like                         float64
is_share                        float64
is_fav                          float64


In [ ]:
ranker_model = tf.saved_model.load('export_saved_model')

In [ ]:
preds=[]
for ind,row in df.iterrows():
      # print("lang ",row['postlang'])
      inputs = {
                  "eng_tags_mask": tf.convert_to_tensor([tf.strings.to_number(tf.strings.split(row['eng_tags_mask'], sep=","),tf.float32)]),
                  "post_dense_features": tf.convert_to_tensor([tf.strings.to_number(tf.strings.split(row['post_dense_features'], sep=","),tf.float32)]),
                  "post_embed": tf.convert_to_tensor([tf.strings.to_number(tf.strings.split(row['post_embed'], sep=","),tf.float32)]),
                  "sparse_features": {"eng_tags": tf.convert_to_tensor([tf.cast(row['sparse_features_eng_tags3'],tf.int32)]),
                                      "tagId": tf.convert_to_tensor([tf.cast(row['sparse_features_tagId'], tf.int32)]),
                                      "userDistrict": tf.convert_to_tensor([tf.cast(row['sparse_features_userDistrict'],tf.int32)])},
                  "time": {"dayofweek": tf.convert_to_tensor([tf.cast(row['time_dayofweek'],tf.int32)]), "hour": tf.convert_to_tensor([tf.cast(row['time_hour'],tf.int32)]), "is_afternoon": tf.convert_to_tensor([tf.expand_dims(tf.cast(row['time_is_afternoon'],tf.float32),axis=-1)]),
                           "is_evening": tf.convert_to_tensor([tf.expand_dims(tf.cast(row['time_is_evening'],tf.float32),axis=-1)]),
                           "is_morning": tf.convert_to_tensor([tf.expand_dims(tf.cast(row['time_is_morning'],tf.float32),axis=-1)]), "is_night": tf.convert_to_tensor([tf.expand_dims(tf.cast(row['time_is_night'],tf.float32),axis=-1)]), "is_weekend": tf.convert_to_tensor([tf.expand_dims(tf.cast(row['time_is_weekend'],tf.float32),axis=-1)])},
                  "user_dense_features": tf.convert_to_tensor([tf.strings.to_number(tf.strings.split(row['user_dense_features'], sep=","),tf.float32)]),
                  "user_embed": tf.convert_to_tensor([tf.strings.to_number(tf.strings.split(row['user_embed'], sep=","),tf.float32)])
                  }
      # print("eng_tags_mask ",inputs["eng_tags_mask"].shape," post_dense_features ",inputs['post_dense_features'].shape," post_embed ",inputs["post_embed"].shape)
      # print("eng_tags ",inputs["sparse_features"]["eng_tags"].shape," tagId ",inputs["sparse_features"]['tagId'].shape," userDistrict ",inputs["sparse_features"]["userDistrict"].shape)
      # print("dayofweek ",inputs["time"]["dayofweek"].shape," hour ",inputs["time"]['hour'].shape," is_afternoon ",inputs["time"]["is_afternoon"].shape," is_evening ",inputs["time"]["is_evening"].shape)
      # print("is_morning ",inputs["time"]["is_morning"].shape," is_night ",inputs["time"]['is_night'].shape," is_weekend ",inputs["time"]["is_weekend"].shape)
      # print("user_dense_features ",inputs["user_dense_features"].shape," user_embed ",inputs['user_embed'].shape)
      preds.append(ranker_model(inputs=inputs,training=False))
      # print("preds ",preds)

In [ ]:
df.drop(['sparse_features_eng_tags2'],axis=1, inplace=True)

In [ ]:
preds2 = [a.numpy()[0] for a in preds]

In [ ]:
df['pred'] = preds2

df.to_csv("final_result_nu.csv",sep='&',index=False)

In [ ]:
df = pd.read_csv("final_result_nu.csv",sep='&')

In [ ]:
df['is_like2'] = df['is_like']
df['is_fav2'] = df['is_fav']
df['is_share2'] = df['is_share']

df['is_like2'].fillna(0,inplace=True)
df['is_share2'].fillna(0,inplace=True)
df['is_fav2'].fillna(0,inplace=True)

In [ ]:
df['is_vp_succ2'] = df['is_vp_succ']+df['is_like2']+df['is_fav2']+df['is_share2']
df['is_vp_succ2'] = df['is_vp_succ2'].apply(lambda x: 1 if x>0 else 0)

In [ ]:


X = df[['isp_score','pred']]
y = df['is_vp_succ2']

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X)
# X = scaler.transform(X)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs')
lr.fit(X,y)

LogisticRegression()

In [ ]:
lr.score(X, y)

0.5932457325310082

In [ ]:
lr.coef_

array([[0.01509657, 0.92485482]])

In [ ]:
df['isp_score_act'] = df[df['model'].notnull()]['isp_score']

In [ ]:
dfk = df[df['isp_score_act'].notnull()]

In [ ]:
X = dfk[['isp_score_act','pred']]
y = dfk['is_vp_succ2']

In [ ]:
lr = LogisticRegression(solver='lbfgs')
lr.fit(X,y)
lr.score(X, y)

In [ ]:
lr.coef_

In [ ]:
tot_score = 0.01509657*X['isp_score_act']+0.92485482*X['pred']

In [ ]:
from sklearn.metrics import mean_squared_error
import math
rms_train = math.sqrt(mean_squared_error(y, tot_score))#dfk['pred']))#df['pred']))#tot_score0.7257371784077328
print("rms_train ",rms_train)

rms_train  0.6705448559833042


In [ ]:
lr_pred_5 = []
for i in tot_score:
    if i>=0.3:
        lr_pred_5.append(1)
    else:
        lr_pred_5.append(0)


In [ ]:
from sklearn.metrics import precision_recall_fscore_support,roc_curve,auc
precision_recall_fscore_support(y,lr_pred_5)

(array([0.69568627, 0.45940669]),
 array([0.03994056, 0.97903496]),
 array([0.07554401, 0.62536456]),
 array([22208, 18507]))

In [ ]:
fpr, tpr, thresholds = roc_curve(y, tot_score, pos_label=1)
auc(fpr, tpr)

0.3966380796077783

In [ ]:
for i in np.arange(0,1,0.01):
  tot_score = i*df['isp_score']+(1-i)*df['pred']
  fpr, tpr, thresholds = roc_curve(df['is_vp_succ'], tot_score, pos_label=1)
  print("i is ",i)
  print("auc ",auc(fpr, tpr))

i is  0.0
auc  0.6037971867268841
i is  0.01
auc  0.6055392505460077
i is  0.02
auc  0.605941962749332
i is  0.03
auc  0.6053609323026994
i is  0.04
auc  0.6041862522438608
i is  0.05
auc  0.602731086565806
i is  0.06
auc  0.6012092286226922
i is  0.07
auc  0.5997587632625939
i is  0.08
auc  0.5984407478718256
i is  0.09
auc  0.5972665276901634
i is  0.1
auc  0.5962343057201602
i is  0.11
auc  0.5953361846369549
i is  0.12
auc  0.5945623222065359
i is  0.13
auc  0.5939065368950642
i is  0.14
auc  0.5933447025330754
i is  0.15
auc  0.5928573480483782
i is  0.16
auc  0.5924277714773558
i is  0.17
auc  0.5920412694197867
i is  0.18
auc  0.5916871164454628
i is  0.19
auc  0.591361507148025
i is  0.2
auc  0.5910632072890271
i is  0.21
auc  0.5907894779451779
i is  0.22
auc  0.5905385703162647
i is  0.23
auc  0.5903073127342301
i is  0.24
auc  0.590093392914803
i is  0.25
auc  0.5898932607099794
i is  0.26
auc  0.5897052656220615
i is  0.27
auc  0.5895297251116939
i is  0.28
auc  0.589364590